In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

import numpy as np
import pandas as pd
import admix
import matplotlib.pyplot as plt
import numpy as np
import json
import pandas as pd
import seaborn as sns
import os
import dapgen
from os.path import join
import admix_genet_cor

In [3]:
GENO_DIR = (
    "/u/project/pasaniuc/pasaniucdata/admixture/projects/PAGE-QC/01-dataset/out/aframr"
)

In [ ]:
pfile = f"{GENO_DIR}/imputed/chr1"

geno, df_snp, df_indiv = dapgen.read_pfile(pfile, phase=True, snp_chunk=2048)
lanc = admix.io.read_lanc(pfile + ".lanc", snp_chunk=2048)
df_snp_info = pd.read_csv(pfile + ".snp_info", sep="\t").set_index("SNP")
assert np.all(df_snp_info.index == df_snp.index.values)
df_snp = pd.merge(df_snp, df_snp_info, left_index=True, right_index=True)

freq_filter = (
    (0.005 <= df_snp.EUR_FREQ)
    & (df_snp.EUR_FREQ <= 0.995)
    & (0.005 <= df_snp.AFR_FREQ)
    & (df_snp.AFR_FREQ <= 0.995)
)

geno = geno[freq_filter.values, :, :]
lanc = lanc[freq_filter.values, :, :]
df_snp = df_snp.iloc[freq_filter.values, :]

In [ ]:
apa = admix.data.allele_per_anc(geno, lanc)
geno_cnt = geno.sum(axis=2)

In [ ]:
window_size = 10000
step_size = 25
n_region = 30

In [ ]:
np.random.seed(1234)
causal_snps = np.random.choice(df_snp.index, size=n_region, replace=False)
causal_idx = [df_snp.index.get_loc(s) for s in causal_snps]

df_cov_diff = {"cov_diff": [], "pos": [], "causal_snp": []}

for snp, idx in zip(causal_snps, causal_idx):
    region_idx = np.arange(idx - window_size, idx + window_size, step_size)
    df_cov_diff["cov_diff"].extend(
        admix_genet_cor.calc_apa_cov(
            apa[region_idx, :, :].compute(),
            geno_cnt[idx, :].compute(),
        )
    )
    # relative base pairs
    df_cov_diff["pos"].extend(
        (df_snp.POS.iloc[region_idx] - df_snp.POS.iloc[idx]).values
    )
    df_cov_diff["causal_snp"].extend([snp] * len(region_idx))
df_cov_diff = pd.DataFrame(df_cov_diff)